In [1]:
pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


In [27]:
from Crypto.PublicKey import RSA

#產生 2048 位元 RSA 金鑰
key = RSA.generate(2048)

# RSA 私鑰
privateKey = key.export_key()
with open("private.pem", "wb") as f:
    f.write(privateKey)

# RSA 公鑰
publicKey = key.publickey().export_key()
with open("public.pem", "wb") as f:
    f.write(publicKey)

In [28]:
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP
import Crypto.Cipher.AES as AES
import operator


with open('name.txt', 'rb') as f:
        data = f.read()

# 讀取 RSA 公鑰
publicKey = RSA.import_key(open("public.pem").read())

# 建立隨機的 AES 金鑰
sessionKey = get_random_bytes(16)

# 以 RSA 金鑰加密 AES 金鑰
cipherRSA = PKCS1_OAEP.new(publicKey)
encSessionKey = cipherRSA.encrypt(sessionKey)

# 以 AES 金鑰加密資料
cipherAES = AES.new(sessionKey, AES.MODE_EAX)
ciphertext, tag = cipherAES.encrypt_and_digest(data)

# 將加密結果寫入檔案
with open("encrypted_data.bin", "wb") as f:
    f.write(encSessionKey)
    f.write(cipherAES.nonce)
    f.write(tag)
    f.write(ciphertext)

In [29]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP

# 讀取 RSA 私鑰
privateKey = RSA.import_key(open("private.pem").read())

# 從檔案讀取加密資料
with open("encrypted_data.bin", "rb") as f:
    encSessionKey = f.read(privateKey.size_in_bytes())
    nonce = f.read(16)
    tag = f.read(16)
    ciphertext = f.read(-1)

# 以 RSA 金鑰解密 AES 金鑰
cipherRSA = PKCS1_OAEP.new(privateKey)
sessionKey = cipherRSA.decrypt(encSessionKey)

# 以 AES 金鑰解密資料
cipherAES = AES.new(sessionKey, AES.MODE_EAX, nonce)
data = cipherAES.decrypt_and_verify(ciphertext, tag)

# 輸出解密後的資料
print(data.decode("utf-8"))

s1092929 徐文献
